In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14752166
paper_name = 'marston_amon_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Table 1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4915 x 7


In [10]:
original_data.head()

,ORF,Gene,% Spores,% Dyads,4,or\n3,or\n2
0,YAL002W,NaN,57,33,76.0,2,22
1,YAL004W,unknown,45,62,75.0,0,25
2,YAL005C,SSA1,67,22,44.0,4,52
3,YAL007C,ERP2,56,57,67.0,0,33
4,YAL008W,FUN14,59,44,96.0,0,4


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
original_data.loc[original_data['orf']=='YMR225','orf'] = 'YMR225C'

In [17]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [18]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             ORF  Gene  % Spores  % Dyads    4  or\n3  or\n2  orf
index_input                                                      
44           ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
91           ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
138          ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
185          ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
232          ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
...          ...   ...       ...      ...  ...    ...    ...  ...
4697         ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
4744         ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
4791         ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
4838         ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF
4885         ORF  Gene  % Spores  % Dyads  4.0  or\n3  or\n2  ORF

[104 rows x 8 columns]


In [19]:
original_data = original_data.loc[t,:]

In [20]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['% Spores','% Dyads', '4']].copy()

In [23]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [24]:
original_data = original_data.groupby(original_data.index).mean()

In [25]:
original_data.shape

(4731, 3)

# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [486, 4963, 4950]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,486,4963,4950
data_type,value,value,value
orf,,,
YAL002W,57.0,33.0,76.0
YAL004W,45.0,62.0,75.0
YAL005C,67.0,22.0,44.0
YAL007C,56.0,57.0,67.0
YAL008W,59.0,44.0,96.0


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,486,4963,4950
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,57.0,33.0,76.0
1863,YAL004W,45.0,62.0,75.0
4,YAL005C,67.0,22.0,44.0
5,YAL007C,56.0,57.0,67.0
6,YAL008W,59.0,44.0,96.0


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,486,4963,4950,486,4963,4950
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,57.0,33.0,76.0,0.292355,0.346008,-0.445657
1863,YAL004W,45.0,62.0,75.0,-0.342596,2.473181,-0.489689
4,YAL005C,67.0,22.0,44.0,0.821481,-0.460850,-1.854680
5,YAL007C,56.0,57.0,67.0,0.239443,2.106427,-0.841944
6,YAL008W,59.0,44.0,96.0,0.398181,1.152867,0.434982


# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 14752166...
Inserting the new data...


100%|██████████| 3/3 [00:20<00:00,  6.77s/it]

Updating the data_modified_on field...
